In [ ]:
import pandas as pd
import numpy as np
import pandas_datareader.data as web
import statsmodels.api as sm
from statsmodels.tsa.regime_switching.markov_regression import MarkovRegression
import matplotlib.pyplot as plt
import yfinance as yf
from utils import *
from models import *

import numpy as np
import random

# Set seed for reproducibility
SEED = 42
np.random.seed(SEED)
random.seed(SEED)


In [ ]:
df = load_macro_data()
returns = load_sp500_excess_returns()
pca = compute_pca(df)
df =merge_macro_excess_returns(df, returns)
horizons = create_horizons(df)

df = add_macro_features(df)

In [ ]:
print(df.head())
print(df.tail())

In [ ]:
df, s_samples_post_yt = estimate_cay_MS_via_gibbs(df, verbose=True)
df, s_samples_post_yt_at = estimate_cay_MS_logyt_logat(df, verbose=True)
df, s_samples_post_pca = estimate_cay_MS_via_gibbs(df, model = 'pca', verbose=True)
print(df.head())

In [ ]:
df, model_fc_yt = estimate_cay_FC_yt(df)
df, model_fc_yt_at = estimate_cay_FC_yt_at(df)
df, model_fc_pca = estimate_cay_FC_pca(df)
print(df)

In [ ]:
ms_model_cay_yt = MarkovRegression(df['cay_FC_yt'], k_regimes=2, trend='c',
                                switching_variance=True)
ms_result_cay_yt = ms_model_cay_yt.fit(em_iter=0)
ms_model_cay_yt_at = MarkovRegression(df['cay_FC_yt_at'], k_regimes=2, trend='c',
                                switching_variance=True)
ms_result_cay_yt_at = ms_model_cay_yt_at.fit(em_iter=0)
ms_model_cay_pca = MarkovRegression(df['pca'], k_regimes=2, trend='c',
                                switching_variance=True, )
ms_result_cay_pca = ms_model_cay_pca.fit(em_iter=0)

print(ms_result_cay_yt.summary())
print(ms_result_cay_yt_at.summary())
print(ms_result_cay_pca.summary())



In [ ]:
macro_controls = ['interest_rate', 'CPI_inflation', 'unemployment']

In [ ]:
ms_model_cay_yt_with_macro = MarkovRegression(df['cay_FC_yt'], k_regimes=2, trend='c', exog=df[macro_controls],
                                switching_variance=True)
ms_result_cay_yt_with_macro = ms_model_cay_yt.fit(em_iter=0)
ms_model_cay_yt_at_with_macro = MarkovRegression(df['cay_FC_yt_at'], k_regimes=2, trend='c', exog=df[macro_controls],
                                switching_variance=True)
ms_result_cay_yt_at_with_macro = ms_model_cay_yt_at.fit(em_iter=0)
ms_model_cay_pca_with_macro = MarkovRegression(df['pca'], k_regimes=2, trend='c', exog=df[macro_controls],
                                switching_variance=True, )
ms_result_cay_pca_with_macro = ms_model_cay_pca.fit(em_iter=0)

print(ms_result_cay_yt.summary())
print(ms_result_cay_yt_at.summary())
print(ms_result_cay_pca.summary())


In [ ]:
fig, axes = plt.subplots(1, 3, figsize=(18, 4), sharex=True)

smoothed_probs = ms_result_cay_yt.smoothed_marginal_probabilities
smoothed_probs.plot(ax=axes[0], title='cay_yt regimes')
axes[0].set_ylabel('Probability')
axes[0].set_xlabel('Date')

smoothed_probs = ms_result_cay_pca.smoothed_marginal_probabilities
smoothed_probs.plot(ax=axes[1], title='cay_pca regimes')
axes[1].set_ylabel('Probability')
axes[1].set_xlabel('Date')

smoothed_probs = ms_result_cay_yt_at.smoothed_marginal_probabilities
smoothed_probs.plot(ax=axes[2], title='cay_yt_at regimes')
axes[2].set_ylabel('Probability')
axes[2].set_xlabel('Date')

plt.tight_layout()
plt.show()

In [ ]:
posterior_yt = run_multi_chain_gibbs(
    df=df,
    gibbs_function=estimate_cay_MS_via_gibbs_final,
    num_chains=4,
    num_iterations=10000,
    burn_in=2000,
    k_regimes=2,
    model='yt',
    verbose=True
)


1 Parameter α₀ (alpha_0)
Left (density): Chains overlap better compared to before but still show multiple peaks (potential label switching or multimodality).

Right (traceplot): Chains show better mixing than before but still some divergence.

➡️ Interpretation: Improved convergence, but chains may still explore separate modes.

🔬 2. Parameter α₁ (alpha_1)
Left: Modes are closer, better overlap.

Right: Chains are more stable but still show drift.

➡️ Interpretation: Partial convergence, better than previous 2k runs.

🔬 3. Parameter β₀ (beta_0)
Left: All chains collapse to a tight high peak at ≈1.

Right: Traceplots are stable with little variation.

➡️ Interpretation: Converged.

🔬 4. Parameter β₁ (beta_1)
Left: Chains have overlapping densities but remain multimodal.

Right: Traceplots drift and do not mix fully.

➡️ Interpretation: Partial convergence, label switching possible.

🔬 5. Parameter σ² (sigma2)
Left: Chains have overlapping densities.

Right: Traceplots stable, no drift.

➡️ Interpretation: Good convergence.

⚠️ Overall Project Convergence Status (10,000 Iterations)
✅ Improved convergence compared to 2k runs
⚠️ Remaining issues:

Label switching (especially in α_s parameters)

Possible model identification challenges

In [ ]:
posterior_pca = run_multi_chain_gibbs(
    df=df,
    gibbs_function=estimate_cay_MS_via_gibbs_final,
    num_chains=4,
    num_iterations=10000,
    burn_in=2000,
    k_regimes=2,
    model='pca',
    verbose=True
)

In [ ]:
import numpy as np
from scipy.stats import invgamma, norm

def estimate_cay_MS_gibbs_macro(df, k_regimes=2, n_iter=1000, burn_in=200, model='yt', verbose=True):
    """
    Gibbs sampler for regime-switching model with macro controls as regime-invariant predictors.
    """
    # ---------------------------
    # 0. Data preparation
    # ---------------------------
    y = df['log_ct'].values
    if model == 'yt':
        X_main = df['log_yt'].values
    elif model == 'pca':
        X_main = df['pca'].values
    else:
        raise ValueError("Invalid model")

    # Macro controls
    Z = df[['interest_rate', 'CPI_inflation', 'unemployment']].values
    T = len(y)
    n_macro = Z.shape[1]

    # ---------------------------
    # 1. Hyperparameters
    # ---------------------------
    sigma2_y = np.var(y)
    mu_alpha = np.zeros(k_regimes)
    sigma2_alpha = np.ones(k_regimes) * sigma2_y
    mu_beta = np.zeros(k_regimes)
    sigma2_beta = np.ones(k_regimes) * sigma2_y

    mu_gamma = np.zeros(n_macro)
    sigma2_gamma = np.ones(n_macro) * sigma2_y

    alpha_sigma = 2.5
    beta_sigma = 0.5
    prior_trans = np.ones((k_regimes, k_regimes)) * 0.5

    # ---------------------------
    # 2. Initialize storage
    # ---------------------------
    alpha_samples = np.zeros((n_iter, k_regimes))
    beta_samples = np.zeros((n_iter, k_regimes))
    gamma_samples = np.zeros((n_iter, n_macro))
    sigma2_samples = np.zeros(n_iter)
    s_samples = np.zeros((n_iter, T))

    # Initialize parameters
    alpha = np.zeros(k_regimes)
    beta = np.zeros(k_regimes)
    gamma = np.zeros(n_macro)
    sigma2 = 1
    P = np.full((k_regimes, k_regimes), 1/k_regimes)
    s_t = np.random.choice(k_regimes, size=T)

    # ---------------------------
    # Helper functions
    # ---------------------------
    def hamilton_filter(y, X_main, Z, alpha, beta, gamma, sigma2, P, pi0):
        T = len(y)
        k = len(alpha)
        xi = np.zeros((T, k))
        for s in range(k):
            mu = alpha[s] + beta[s]*X_main[0] + np.dot(Z[0,:], gamma)
            xi[0, s] = pi0[s] * norm.pdf(y[0], loc=mu, scale=np.sqrt(sigma2))
        xi[0, :] /= xi[0, :].sum()
        for t in range(1, T):
            for s in range(k):
                mu = alpha[s] + beta[s]*X_main[t] + np.dot(Z[t,:], gamma)
                xi[t, s] = norm.pdf(y[t], loc=mu, scale=np.sqrt(sigma2)) * np.dot(xi[t-1, :], P[:, s])
            xi[t, :] /= xi[t, :].sum()
        return xi

    def backward_sampling(xi, P):
        T, k = xi.shape
        s_t = np.zeros(T, dtype=int)
        s_t[T-1] = np.random.choice(k, p=xi[T-1, :])
        for t in range(T-2, -1, -1):
            prob = xi[t, :] * P[:, s_t[t+1]]
            prob /= prob.sum()
            s_t[t] = np.random.choice(k, p=prob)
        return s_t

    # ---------------------------
    # 3. Gibbs sampler loop
    # ---------------------------
    for it in range(n_iter):
        # Step 1. Sample s_t
        pi0 = np.full(k_regimes, 1/k_regimes)
        xi = hamilton_filter(y, X_main, Z, alpha, beta, gamma, sigma2, P, pi0)
        s_t = backward_sampling(xi, P)

        # Step 2. Sample alpha_s and beta_s (regime-dependent)
        for s in range(k_regimes):
            idx = (s_t == s)
            n_s = np.sum(idx)
            if n_s > 0:
                y_s = y[idx]
                X_s = X_main[idx]
                Z_s = Z[idx,:]
                y_tilde = y_s - np.dot(Z_s, gamma)

                # Sample beta_s
                var_beta = 1 / (np.sum(X_s**2) / sigma2 + 1 / sigma2_beta[s])
                mean_beta = var_beta * (np.sum(X_s * (y_tilde - alpha[s])) / sigma2)
                beta[s] = norm.rvs(loc=mean_beta, scale=np.sqrt(var_beta))

                # Sample alpha_s
                var_alpha = 1 / (n_s / sigma2 + 1 / sigma2_alpha[s])
                mean_alpha = var_alpha * (np.sum(y_tilde - beta[s] * X_s) / sigma2)
                alpha[s] = norm.rvs(loc=mean_alpha, scale=np.sqrt(var_alpha))
            else:
                beta[s] = norm.rvs(loc=mu_beta[s], scale=np.sqrt(sigma2_beta[s]))
                alpha[s] = norm.rvs(loc=mu_alpha[s], scale=np.sqrt(sigma2_alpha[s]))

        # Enforce ordering constraint (α_0 < α_1)
        sort_idx = np.argsort(alpha)
        alpha = alpha[sort_idx]
        beta = beta[sort_idx]
        new_s_t = np.zeros_like(s_t)
        for new_label, old_label in enumerate(sort_idx):
            new_s_t[s_t == old_label] = new_label
        s_t = new_s_t

        # Step 3. Sample gamma (regime-invariant macro coefficients)
        ZTZ = np.dot(Z.T, Z)
        var_gamma = np.linalg.inv(ZTZ / sigma2 + np.diag(1 / sigma2_gamma))
        y_resid = y - (alpha[s_t] + beta[s_t] * X_main)
        mean_gamma = np.dot(var_gamma, np.dot(Z.T, y_resid) / sigma2)
        gamma = np.random.multivariate_normal(mean_gamma, var_gamma)

        # Step 4. Sample sigma2
        resid = y - (alpha[s_t] + beta[s_t]*X_main + np.dot(Z, gamma))
        alpha_post = alpha_sigma + T/2
        beta_post = beta_sigma + 0.5 * np.sum(resid**2)
        sigma2 = invgamma.rvs(a=alpha_post, scale=beta_post)

        # Step 5. Sample transition matrix rows
        counts = np.zeros((k_regimes, k_regimes))
        for t in range(T-1):
            counts[s_t[t], s_t[t+1]] += 1
        for s in range(k_regimes):
            P[s,:] = np.random.dirichlet(prior_trans[s,:] + counts[s,:])

        # Store samples
        alpha_samples[it,:] = alpha
        beta_samples[it,:] = beta
        gamma_samples[it,:] = gamma
        sigma2_samples[it] = sigma2
        s_samples[it,:] = s_t

        if verbose and (it+1) % 100 == 0:
            print(f"Iteration {it+1} complete")

    # ---------------------------
    # 4. Post-processing
    # ---------------------------
    results = {
        'alpha_samples': alpha_samples[burn_in:],
        'beta_samples': beta_samples[burn_in:],
        'gamma_samples': gamma_samples[burn_in:],
        'sigma2_samples': sigma2_samples[burn_in:],
        's_samples': s_samples[burn_in:]
    }

    if verbose:
        print(f"Gibbs sampling with macro controls completed for model {model}")

    return results


In [ ]:
posterior_yt = run_multi_chain_gibbs(
    df=df,
    gibbs_function=estimate_cay_MS_gibbs_macro,
    num_chains=4,
    num_iterations=10000,
    burn_in=2000,
    k_regimes=2,
    model='yt',
    verbose=True
)


In [ ]:
posterior_yt = run_multi_chain_gibbs(
    df=df,
    gibbs_function=estimate_cay_MS_gibbs_macro,
    num_chains=4,
    num_iterations=10000,
    burn_in=2000,
    k_regimes=2,
    model='pca',
    verbose=True
)

✅ Interpretation of Your PCA Model Traceplots and Density Plots
Here is a structured evaluation for this new run:

🔬 1. Parameter α₀ (alpha_0)
Left (density): Chains overlap perfectly, unimodal Gaussian shape.

Right (traceplot): Chains mix well, no drift, stable mean.

➡️ Interpretation: Excellent convergence.

🔬 2. Parameter α₁ (alpha_1)
Left: Similar to α₀, smooth single mode, tight overlap.

Right: Stable traceplots, good mixing.

➡️ Interpretation: Excellent convergence.

🔬 3. Parameter β₀ (beta_0)
Left: Overlapping Gaussian densities across chains.

Right: Traceplots stationary with full mixing.

➡️ Interpretation: Excellent convergence.

🔬 4. Parameter β₁ (beta_1)
Left: Perfect overlap, unimodal.

Right: Stable traceplots with high frequency variation indicating good mixing.

➡️ Interpretation: Excellent convergence.

🔬 5. Parameter σ² (sigma2)
Left: Gaussian shape, all chains identical.

Right: Traceplots stable, good mixing.

➡️ Interpretation: Excellent convergence.

✅ Overall PCA Model Convergence Diagnosis
✅ All parameters show full convergence
✅ No label switching or multimodality issues detected
✅ Chains mix well and are stationary

In [ ]:
print(outputs_yt['df'].head())

In [ ]:
outputs_pca = estimate_cay_MS_via_gibbs_final(df, model='pca', n_iter=10000, burn_in=2000, verbose=True)
outputs_pca_macro = estimate_cay_MS_gibbs_macro(df, k_regimes=2, n_iter=1000, burn_in=200, model='pca', verbose=True)
outputs_yt = estimate_cay_MS_via_gibbs_final(df, model='yt', n_iter=10000, burn_in=2000, verbose=True)
outputs_yt_macro = estimate_cay_MS_gibbs_macro(df, k_regimes=2, n_iter=1000, burn_in=200, model='yt', verbose=True)


In [ ]:
compare_cay_FC_vs_MS(outputs_pca['df'], horizons, model='pca')
compare_cay_FC_vs_MS(outputs_yt['df'], horizons, model='yt')

In [ ]:
compare_cay_FC_vs_MS_with_macro(outputs_yt['df'], horizons, model='yt')
compare_cay_FC_vs_MS_with_macro(outputs_pca['df'], horizons, model='pca')

In [ ]:
import matplotlib.pyplot as plt
import numpy as np

# === Extract MLE smoothed probabilities ===

# Assuming 2 regimes
mle_probs_pca = ms_result_cay_pca.smoothed_marginal_probabilities

mle_regime0_pca = mle_probs_pca[0].values
mle_regime1_pca = mle_probs_pca[1].values

# === Compute Bayesian posterior regime probabilities ===

# s_samples_pca: (iterations, T)
bayesian_regime0_pca = (outputs_pca['s_samples'] == 0).mean(axis=0)
bayesian_regime1_pca = (outputs_pca['s_samples'] == 1).mean(axis=0)

# === Plot comparison ===

plt.figure(figsize=(14, 6))

# Regime 0
plt.subplot(2,1,1)
plt.plot(mle_regime0_pca, label='MLE Regime 0', alpha=0.7)
plt.plot(bayesian_regime0_pca, label='Bayesian Regime 0', alpha=0.7)
plt.title('PCA Regime 0 Probability Comparison')
plt.legend()

# Regime 1
plt.subplot(2,1,2)
plt.plot(mle_regime1_pca, label='MLE Regime 1', alpha=0.7)
plt.plot(bayesian_regime1_pca, label='Bayesian Regime 1', alpha=0.7)
plt.title('PCA Regime 1 Probability Comparison')
plt.legend()

plt.tight_layout()
plt.show()

In [ ]:
import matplotlib.pyplot as plt
import numpy as np

# === Extract MLE smoothed probabilities ===

# Assuming 2 regimes
mle_probs_yt = ms_result_cay_yt.smoothed_marginal_probabilities

mle_regime0_yt = mle_probs_yt[0].values
mle_regime1_yt = mle_probs_yt[1].values

# === Compute Bayesian posterior regime probabilities ===

# s_samples_pca: (iterations, T)
bayesian_regime0_yt = (outputs_yt['s_samples'] == 0).mean(axis=0)
bayesian_regime1_yt = (outputs_yt['s_samples'] == 1).mean(axis=0)

# === Plot comparison ===

plt.figure(figsize=(14, 6))

# Regime 0
plt.subplot(2,1,1)
plt.plot(mle_regime0_yt, label='MLE Regime 0', alpha=0.7)
plt.plot(bayesian_regime0_yt, label='Bayesian Regime 0', alpha=0.7)
plt.title('PCA Regime 0 Probability Comparison')
plt.legend()

# Regime 1
plt.subplot(2,1,2)
plt.plot(mle_regime1_yt, label='MLE Regime 1', alpha=0.7)
plt.plot(bayesian_regime1_yt, label='Bayesian Regime 1', alpha=0.7)
plt.title('PCA Regime 1 Probability Comparison')
plt.legend()

plt.tight_layout()
plt.show()

In [ ]:
import matplotlib.pyplot as plt
import numpy as np

# === Extract MLE smoothed probabilities ===

# Assuming 2 regimes
mle_probs_pca_with_macro = ms_result_cay_pca_with_macro.smoothed_marginal_probabilities

mle_regime0_pca = mle_probs_pca_with_macro[0].values
mle_regime1_pca = mle_probs_pca_with_macro[1].values

# === Compute Bayesian posterior regime probabilities ===

# s_samples_pca: (iterations, T)
bayesian_regime0_pca = (outputs_pca_macro['s_samples'] == 0).mean(axis=0)
bayesian_regime1_pca = (outputs_pca_macro['s_samples'] == 1).mean(axis=0)

# === Plot comparison ===

plt.figure(figsize=(14, 6))

# Regime 0
plt.subplot(2,1,1)
plt.plot(mle_regime0_pca, label='MLE Regime 0', alpha=0.7)
plt.plot(bayesian_regime0_pca, label='Bayesian Regime 0', alpha=0.7)
plt.title('PCA Regime 0 Probability Comparison')
plt.legend()

# Regime 1
plt.subplot(2,1,2)
plt.plot(mle_regime1_pca, label='MLE Regime 1', alpha=0.7)
plt.plot(bayesian_regime1_pca, label='Bayesian Regime 1', alpha=0.7)
plt.title('PCA Regime 1 Probability Comparison')
plt.legend()

plt.tight_layout()
plt.show()

In [ ]:
import matplotlib.pyplot as plt
import numpy as np

# === Extract MLE smoothed probabilities ===

# Assuming 2 regimes
mle_probs_yt_with_macro = ms_result_cay_yt_with_macro.smoothed_marginal_probabilities

mle_regime0_yt_with_macro = mle_probs_yt_with_macro[0].values
mle_regime1_yt_with_macro = mle_probs_yt_with_macro[1].values

# === Compute Bayesian posterior regime probabilities ===

# s_samples_pca: (iterations, T)
bayesian_regime0_yt_with_macro = (outputs_yt_macro['s_samples'] == 0).mean(axis=0)
bayesian_regime1_yt_with_macro = (outputs_yt_macro['s_samples'] == 1).mean(axis=0)

# === Plot comparison ===

plt.figure(figsize=(14, 6))

# Regime 0
plt.subplot(2,1,1)
plt.plot(mle_regime0_yt_with_macro, label='MLE Regime 0', alpha=0.7)
plt.plot(bayesian_regime0_yt_with_macro, label='Bayesian Regime 0', alpha=0.7)
plt.title('Yt Regime 0 Probability Comparison')
plt.legend()

# Regime 1
plt.subplot(2,1,2)
plt.plot(mle_regime1_yt_with_macro, label='MLE Regime 1', alpha=0.7)
plt.plot(bayesian_regime1_yt_with_macro, label='Bayesian Regime 1', alpha=0.7)
plt.title('Yt Regime 1 Probability Comparison')
plt.legend()

plt.tight_layout()
plt.show()

In [ ]:
import matplotlib.pyplot as plt
import numpy as np

# === Extract MLE smoothed probabilities ===

# Assuming 2 regimes
mle_probs_yt = ms_result_cay_yt.smoothed_marginal_probabilities

mle_regime0_yt = mle_probs_yt[0].values
mle_regime1_yt = mle_probs_yt[1].values

# === Compute Bayesian posterior regime probabilities ===

# s_samples_pca: (iterations, T)
bayesian_regime0_yt = (outputs_yt['s_samples'] == 0).mean(axis=0)
bayesian_regime1_yt = (outputs_yt['s_samples'] == 1).mean(axis=0)

# === Plot comparison ===

plt.figure(figsize=(14, 6))

# Regime 0
plt.subplot(2,1,1)
plt.plot(mle_regime0_yt, label='MLE Regime 0', alpha=0.7)
plt.plot(bayesian_regime0_yt, label='Bayesian Regime 0', alpha=0.7)
plt.title('Yt Regime 0 Probability Comparison')
plt.legend()

# Regime 1
plt.subplot(2,1,2)
plt.plot(mle_regime1_yt, label='MLE Regime 1', alpha=0.7)
plt.plot(bayesian_regime1_yt, label='Bayesian Regime 1', alpha=0.7)
plt.title('Yt Regime 1 Probability Comparison')
plt.legend()

plt.tight_layout()
plt.show()

In [ ]:
# Select relevant columns
macro_controls = ['interest_rate', 'CPI_inflation', 'unemployment']

# Drop rows with missing values to ensure consistent regression dataset
df_macro = df[['future_ret_1q', 'cay_FC_yt'] + macro_controls].dropna()

# Extract target and predictors
Y = df_macro['future_ret_1q']
X = df_macro[['cay_FC_yt'] + macro_controls]

# Add constant for intercept
import statsmodels.api as sm
X = sm.add_constant(X)


In [ ]:
# Fit OLS regression
model = sm.OLS(Y, X)
results = model.fit()

# Print summary
print(results.summary())


In [ ]:
print(outputs_yt['df'].info())

In [ ]:
# Assuming df_ms prepared as above
Y_ms = df['future_ret_1q']
X_ms = outputs_yt['df'][['cay_MS_yt'] + macro_controls].dropna()
X_ms = sm.add_constant(X_ms)

model_ms = sm.OLS(Y_ms, X_ms)
results_ms = model_ms.fit()

print(results_ms.summary())


In [ ]:
compare_cay_FC_vs_MS_with_macro(outputs_yt['df'], horizons, model='yt')

In [ ]:
from statsmodels.stats.outliers_influence import variance_inflation_factor

# Calculate VIF for each variable
vif_data = pd.DataFrame()
vif_data["variable"] = X_ms.columns
vif_data["VIF"] = [variance_inflation_factor(X.values, i)
                   for i in range(X.shape[1])]

print(vif_data)


In [ ]:
from statsmodels.stats.outliers_influence import variance_inflation_factor

# Calculate VIF for each variable
vif_data = pd.DataFrame()
vif_data["variable"] = X.columns
vif_data["VIF"] = [variance_inflation_factor(X.values, i)
                   for i in range(X.shape[1])]
print(vif_data)


In [ ]:
import seaborn as sns

import matplotlib.pyplot as plt

# Compute correlation matrix for df_macro
corr = df_macro.corr()

plt.figure(figsize=(8, 6))
sns.heatmap(corr, annot=True, cmap='coolwarm', fmt=".2f", square=True)
plt.title('Correlation Matrix (df_macro)')
plt.tight_layout()
plt.show()

In [ ]:
compare_cay_FC_vs_MS(df, horizons, model='yt')
compare_cay_FC_vs_MS(df, horizons, model='yt_at')
compare_cay_FC_vs_MS(df, horizons, model='pca')

In [ ]:
from utils import load_all_cay_FC_vs_MS_results

all_results = load_all_cay_FC_vs_MS_results()

In [ ]:

# Calculate posterior regime probabilities
regime_probs = np.mean(outputs_yt['s_samples'], axis=0)

# Slice regime_probs to match df
regime_probs_matched = regime_probs[-len(df.index):]

plt.figure(figsize=(12,4))
plt.plot(df.index, regime_probs_matched, label='Regime 1 Posterior Probability with yt_at')
plt.title('Estimated Regime 1 Posterior Probability over Time')
plt.ylabel('Probability')
plt.xlabel('Date')
plt.legend()
plt.show()

In [ ]:

# Calculate posterior regime probabilities
regime_probs = np.mean(outputs_pca['s_samples'], axis=0)

# Plot
# Slice regime_probs to match df
regime_probs_matched = regime_probs[-len(df.index):]

plt.figure(figsize=(12,4))
plt.plot(df.index, regime_probs_matched, label='Regime 1 Posterior Probability with yt')
plt.title('Estimated Regime 1 Posterior Probability over Time')
plt.ylabel('Probability')
plt.xlabel('Date')
plt.legend()
plt.show()

In [ ]:

# Calculate posterior regime probabilities
regime_probs = np.mean(s_samples_post_yt_at, axis=0)

# Slice regime_probs to match df
regime_probs_matched = regime_probs[-len(df.index):]

plt.figure(figsize=(12,4))
plt.plot(df.index, regime_probs_matched, label='Regime 1 Posterior Probability with yt_at')
plt.title('Estimated Regime 1 Posterior Probability over Time')
plt.ylabel('Probability')
plt.xlabel('Date')
plt.legend()
plt.show()

In [ ]:
# Assuming smoothed_probs is your regime probability DataFrame
# regime_probs_df = smoothed_probs[[1]].rename(columns={1: 'regime1_prob'})
# df = df.merge(regime_probs_df, left_index=True, right_index=True, how='left')

In [ ]:
X = df_reg[[cay_var, 'interest_rate', 'CPI_inflation']]
X = sm.add_constant(X)


In [ ]:
import numpy as np
from scipy.stats import norm, invgamma

def estimate_cay_MS_via_gibbs_version_3(df, k_regimes=2, n_iter=1000, burn_in=200, model='yt', verbose=True):
    """
    Estimate cay_MS using a Markov-switching regression with Gibbs sampling.
    Implements Bianchi et al. (2016) recommendations:
    - Calibrated Dirichlet priors
    - Ordering constraint (α_0 < α_1)
    Returns: dict with df and posterior samples
    """

    # ---------------------------
    # 0. Data preparation
    # ---------------------------
    if model == 'yt':
        y = df['log_ct'].values
        X = df['log_yt'].values
    elif model == 'pca':
        y = df['log_ct'].values
        X = df['pca'].values
    else:
        raise ValueError(f"Unknown model type: {model}")

    T = len(y)

    # ---------------------------
    # 1. Hyperparameters
    # ---------------------------
    sigma2_y = np.var(y)
    mu_alpha = np.zeros(k_regimes)
    sigma2_alpha = np.ones(k_regimes) * sigma2_y
    mu_beta = 0
    sigma2_beta = sigma2_y
    alpha_sigma = 2.5
    beta_sigma = 0.5
    prior_trans = np.ones((k_regimes, k_regimes)) * 2  # Beta(2,2)-equivalent

    # ---------------------------
    # 2. Initialize storage
    # ---------------------------
    alpha_samples = np.zeros((n_iter, k_regimes))
    beta_samples = np.zeros(n_iter)
    sigma2_samples = np.zeros(n_iter)
    s_samples = np.zeros((n_iter, T))

    # Initialize parameters
    alpha = np.zeros(k_regimes)
    beta = 0
    sigma2 = 1
    P = np.full((k_regimes, k_regimes), 1/k_regimes)
    s_t = np.random.choice(k_regimes, size=T)

    # ---------------------------
    # Helper functions
    # ---------------------------
    def hamilton_filter(y, X, alpha, beta, sigma2, P, pi0):
        T = len(y)
        k = len(alpha)
        xi = np.zeros((T, k))
        for s in range(k):
            mu = alpha[s] + beta * X[0]
            xi[0, s] = pi0[s] * norm.pdf(y[0], loc=mu, scale=np.sqrt(sigma2))
        xi[0, :] /= xi[0, :].sum()
        for t in range(1, T):
            for s in range(k):
                mu = alpha[s] + beta * X[t]
                xi[t, s] = norm.pdf(y[t], loc=mu, scale=np.sqrt(sigma2)) * np.dot(xi[t-1, :], P[:, s])
            xi[t, :] /= xi[t, :].sum()
        return xi

    def backward_sampling(xi, P):
        T, k = xi.shape
        s_t = np.zeros(T, dtype=int)
        s_t[T-1] = np.random.choice(k, p=xi[T-1, :])
        for t in range(T-2, -1, -1):
            prob = xi[t, :] * P[:, s_t[t+1]]
            prob /= prob.sum()
            s_t[t] = np.random.choice(k, p=prob)
        return s_t

    # ---------------------------
    # 5. Gibbs sampler loop
    # ---------------------------
    for it in range(n_iter):
        # Step 1. Sample s_t using Hamilton filter + backward sampling
        pi0 = np.full(k_regimes, 1/k_regimes)
        xi = hamilton_filter(y, X, alpha, beta, sigma2, P, pi0)
        s_t = backward_sampling(xi, P)

        # Step 2. Sample alpha_s with ordering constraint
        for s in range(k_regimes):
            idx = (s_t == s)
            n_s = np.sum(idx)
            if n_s > 0:
                y_s = y[idx]
                X_s = X[idx]
                var_alpha = 1 / (n_s / sigma2 + 1 / sigma2_alpha[s])
                var_alpha = max(var_alpha, 1e-8)
                mean_alpha = var_alpha * (np.sum(y_s - beta * X_s) / sigma2)
                alpha[s] = norm.rvs(loc=mean_alpha, scale=np.sqrt(var_alpha))
            else:
                alpha[s] = norm.rvs(loc=mu_alpha[s], scale=np.sqrt(sigma2_alpha[s]))

        # Enforce ordering constraint (α_0 < α_1)
        sort_idx = np.argsort(alpha)
        alpha = alpha[sort_idx]
        new_s_t = np.zeros_like(s_t)
        for new_label, old_label in enumerate(sort_idx):
            new_s_t[s_t == old_label] = new_label
        s_t = new_s_t

        # Step 3. Sample beta
        y_tilde = y - alpha[s_t]
        var_beta = 1 / (np.sum(X**2) / sigma2 + 1 / sigma2_beta)
        var_beta = max(var_beta, 1e-8)
        mean_beta = var_beta * (np.sum(X * y_tilde) / sigma2)
        beta = norm.rvs(loc=mean_beta, scale=np.sqrt(var_beta))

        # Step 4. Sample sigma2
        resid = y - alpha[s_t] - beta * X
        alpha_post = alpha_sigma + T/2
        beta_post = beta_sigma + 0.5 * np.sum(resid**2)
        sigma2 = invgamma.rvs(a=alpha_post, scale=beta_post)

        # Step 5. Sample transition matrix rows
        counts = np.zeros((k_regimes, k_regimes))
        for t in range(T-1):
            counts[s_t[t], s_t[t+1]] += 1
        for s in range(k_regimes):
            P[s,:] = np.random.dirichlet(prior_trans[s,:] + counts[s,:])

        # Store samples
        alpha_samples[it,:] = alpha
        beta_samples[it] = beta
        sigma2_samples[it] = sigma2
        s_samples[it,:] = s_t

        if verbose and (it+1) % 100 == 0:
            print(f"Iteration {it+1} complete")

    # ---------------------------
    # 6. Post-processing
    # ---------------------------
    alpha_samples_ = alpha_samples[burn_in:]
    beta_samples_ = beta_samples[burn_in:]
    sigma2_samples_ = sigma2_samples[burn_in:]
    s_samples_ = s_samples[burn_in:]

    alpha_mean = alpha_samples_.mean(axis=0)
    beta_mean = beta_samples_.mean()
    s_t_mode = np.round(s_samples_.mean(axis=0)).astype(int)
    cay_MS = y - (alpha_mean[s_t_mode] + beta_mean * X)

    # Add to dataframe
    df = df.copy()
    df[f'cay_MS_{model}'] = cay_MS

    if verbose:
        print(f"Full Gibbs sampling with Hamilton filter completed. cay_MS for model {model} calculated and added to dataframe.")

    # Return results dict
    return {
        'df': df,
        'alpha_samples': alpha_samples_,
        'beta_samples': beta_samples_,
        'sigma2_samples': sigma2_samples_,
        's_samples': s_samples_
    }


In [ ]:
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd

def summarize_gibbs_results(df, s_samples_post, alpha_samples, beta_samples, sigma2_samples, model_label='yt'):
    """
    Summarize Gibbs sampler outputs:
    - Plots posterior regime probabilities
    - Returns parameter summary dataframe
    """

    results = {}

    # ---------------------------
    # 1. Calculate posterior regime probabilities
    # ---------------------------
    regime_probs = np.mean(s_samples_post, axis=0)

    # Align regime_probs with df index
    if len(regime_probs) > len(df.index):
        regime_probs_matched = regime_probs[-len(df.index):]
    elif len(regime_probs) < len(df.index):
        raise ValueError("regime_probs length is shorter than df index. Check input alignment.")
    else:
        regime_probs_matched = regime_probs

    # Plot regime probabilities
    plt.figure(figsize=(12,4))
    plt.plot(df.index, regime_probs_matched, label=f'Regime 1 Posterior Probability with {model_label}')
    plt.title(f'Estimated Regime 1 Posterior Probability over Time ({model_label})')
    plt.ylabel('Probability')
    plt.xlabel('Date')
    plt.legend()
    plt.tight_layout()
    plt.show()

    # Save to results dict
    results['regime_probs'] = regime_probs_matched

    # ---------------------------
    # 2. Parameter posterior summaries
    # ---------------------------
    alpha_mean = alpha_samples.mean(axis=0)
    alpha_std = alpha_samples.std(axis=0)
    beta_mean = beta_samples.mean()
    beta_std = beta_samples.std()
    sigma2_mean = sigma2_samples.mean()
    sigma2_std = sigma2_samples.std()

    # Combine into a dataframe
    param_summary = pd.DataFrame({
        'Parameter': [f'alpha_{i}' for i in range(len(alpha_mean))] + ['beta', 'sigma2'],
        'Mean': np.concatenate([alpha_mean, [beta_mean, sigma2_mean]]),
        'Std': np.concatenate([alpha_std, [beta_std, sigma2_std]])
    })

    print("\n🔷 Posterior Parameter Summary:")
    print(param_summary)

    # Save to results dict
    results['param_summary'] = param_summary

    # ---------------------------
    # 3. Effective sample size diagnostics (optional)
    # ---------------------------
    # Example: simple ESS estimate (inefficient but quick)
    def ess(x):
        n = len(x)
        acf_sum = 0
        for lag in range(1, min(1000, n)):
            acf = np.corrcoef(x[:-lag], x[lag:])[0,1]
            if np.isnan(acf) or acf < 0:
                break
            acf_sum += 2 * acf
        return n / (1 + acf_sum)

    beta_ess = ess(beta_samples)
    sigma2_ess = ess(sigma2_samples)

    print(f"\n🔷 Effective Sample Size (ESS): beta = {beta_ess:.1f}, sigma2 = {sigma2_ess:.1f}")

    results['ess_beta'] = beta_ess
    results['ess_sigma2'] = sigma2_ess

    # Return results dictionary
    return results


In [ ]:
outputs = estimate_cay_MS_via_gibbs_version_3(df, model='yt', verbose=True)

summary = summarize_gibbs_results(
    df = outputs['df'],
    s_samples_post = outputs['s_samples'],
    alpha_samples = outputs['alpha_samples'],
    beta_samples = outputs['beta_samples'],
    sigma2_samples = outputs['sigma2_samples'],
    model_label = 'yt'
)


In [ ]:
outputs = estimate_cay_MS_via_gibbs_version_3(df, model='pca', verbose=True)

summary = summarize_gibbs_results(
    df = outputs['df'],
    s_samples_post = outputs['s_samples'],
    alpha_samples = outputs['alpha_samples'],
    beta_samples = outputs['beta_samples'],
    sigma2_samples = outputs['sigma2_samples'],
    model_label = 'pca'
)

In [ ]:
import numpy as np

def compute_implied_sharpe(y_true, y_pred, freq=4):
    """
    Computes the implied Sharpe ratio from regression forecast residuals.

    Parameters:
    - y_true: array-like, true realized returns
    - y_pred: array-like, predicted returns from regression
    - freq: int, annualization factor (e.g. 4 for quarterly data)

    Returns:
    - sr_annualized: annualized Sharpe ratio
    """
    # Compute residuals
    residuals = y_true - y_pred

    # Standard deviation of true returns and residuals
    sigma_r = np.std(y_true, ddof=1)
    sigma_e = np.std(residuals, ddof=1)

    # R-squared
    r_squared = 1 - (np.var(residuals, ddof=1) / np.var(y_true, ddof=1))

    # Implied Sharpe ratio (unannualized)
    sr = np.sqrt(max(r_squared,0))

    # Annualize
    sr_annualized = sr * np.sqrt(freq)

    print(f"R²: {r_squared:.4f}")
    print(f"Implied unannualized SR: {sr:.4f}")
    print(f"Implied annualized SR: {sr_annualized:.4f}")

    return sr_annualized

# === Example usage ===

# Replace with your actual regression inputs
# y_true = df['future_ret_4q'].values
# y_pred = regression_model.predict(df[['const', 'cay_FC_yt']])

# sr = compute_implied_sharpe(y_true, y_pred, freq=4)
